<h1> Test the implemented  model </h1>

<i> this is a simple jupyter notebook on how to run different examples using the created model</i>
<ul>
    <li> execute each of the following cells one time using CTRL + ENTER </li>
    <li> in the third cell just give the path of the image to be tested </li>
    <li> execute the last cell and as an output an array which contains all the detected sudoku puzzle digits will be printed </li>  
    
</ul>

In [19]:
# load the saved model for testing 

from keras.models import load_model

model = load_model('model_sudoku.h5')

In [23]:
# rectify and split_test_image_puzzle are responsible for splitting the sudoku puzzle to cells each cell contains one digit of the puzzle

import cv2
import numpy as np 
import pytesseract
from skimage.segmentation import clear_border



def rectify(h):
    
        h = h.reshape((4,2))
        hnew = np.zeros((4,2),dtype = np.float32)

        add = h.sum(1)
        hnew[0] = h[np.argmin(add)]
        hnew[2] = h[np.argmax(add)]

        diff = np.diff(h,axis = 1)
        hnew[1] = h[np.argmin(diff)]
        hnew[3] = h[np.argmax(diff)]

        return hnew


# detect the puzzle and split it into 81 images cells digit 

def split_test_image_puzzle(path : str):
    
        img=cv2.imread(path)  # give the path file of the test image 

        img = cv2.resize(img,(450,450))
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        gray = cv2.GaussianBlur(gray,(5,5),0)
        thresh = cv2.adaptiveThreshold(gray,255,1,1,11,2)


        contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        biggest = None
        max_area = 0
        for i in contours:
                area = cv2.contourArea(i)
                if area > 100:
                        peri = cv2.arcLength(i,True)
                        approx = cv2.approxPolyDP(i,0.02*peri,True)
                        if area > max_area and len(approx)==4:
                                biggest = approx
                                max_area = area
        cv2.drawContours(img, biggest, -1, (0,255,0), 8)



        biggest=rectify(biggest)

        h = np.array([ [0,0],[449,0],[449,449],[0,449] ],np.float32)
        retval = cv2.getPerspectiveTransform(biggest,h)
        thresh = cv2.adaptiveThreshold(gray,255,1,1,11,2)

        warp = cv2.warpPerspective(thresh,retval,(450,450))   # contains the sudoku puzzle


        winX = int(warp.shape[1]/9.0)
        winY = int(warp.shape[0]/9.0)
 
        liste3 = [] 

        #loop through all the cells of the sudoku puzzle and append each digit's image to the list

        for y in range(0,warp.shape[0],winY):  
            for x in range(0,warp.shape[1],winX):
                #slice the cell
                window = warp[y:y+winY,x:x+winX]
                #sanity check
                if window.shape[0] != winY or window.shape[1] != winX:
                    continue
                clone = warp.copy()
                digit = cv2.resize(window,(28,28))

                #clear borders


                digit = clear_border(digit)  # contains one cell from the puzzle 

                # append the digit image which corresponds to one cell in the puzzle to a list 

                liste3.append(digit.reshape(28,28,1)) 

        
        return liste3

In [24]:
#load the test image 
test = split_test_image_puzzle('test_examples_images/image91.jpg')  # give the path of the image to be tested.
test = np.asarray(test)
test = test.astype(np.float32) / 255

In [25]:
# predict and recognize each digit of our sudoku puzzle using the saved model and append the predicted value to the final output array.

liste = model.predict_classes(test)
data = liste.tolist()
print ([data[x:x+9] for x in range(0, len(data), 9)])

[[1, 0, 0, 0, 0, 0, 0, 6, 0], [0, 2, 6, 5, 0, 1, 0, 0, 0], [3, 0, 0, 8, 0, 9, 0, 0, 0], [0, 0, 0, 0, 9, 0, 7, 1, 0], [0, 0, 2, 0, 0, 0, 9, 0, 0], [0, 1, 5, 0, 7, 0, 0, 0, 0], [0, 0, 0, 4, 0, 2, 0, 0, 9], [0, 0, 0, 6, 0, 7, 8, 2, 0], [0, 8, 0, 0, 0, 0, 0, 0, 3]]
